In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
# import requests
# import pymongo
from splinter import Browser
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
import datetime
from dateutil.parser import parse

In [2]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
url2 = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
url3 = 'https://space-facts.com/mars/'
url4 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [3]:
#prepared for run splinter, initiation
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [4]:
#Visite the required url
browser.visit(url)
time.sleep(1)

In [5]:
 # Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [6]:
# Get the news' title and paragraph
news = []

threads = soup.find_all('li', class_='slide')

for thread in threads:
    div = thread.find('div', class_= 'list_text')
    temp = {}
    try: 
        dt = div.find('div', class_= 'list_date').text
        d = parse(dt).strftime("%y-%m-%d")
    
        if d >= '21-03-05':
            p = div.find('div', class_= 'article_teaser_body').text
            if div.a.text:
                t = div.a.text
                if t.split("-"):
                    i = t.split("-")
                    t = i[0]
            if (d and t and p):
                temp.update({'date': d, 'title': t, 'paragraph': p})
                news.append(temp)
                
    except AttributeError as e:
        print(e)
               

In [7]:
# for n in news:
#     print(n)

In [8]:
#Visite the required url2
browser.visit(url2)
time.sleep(1)

In [9]:
 # Scrape page into Soup
html2 = browser.html
soup = bs(html2, "html.parser")

In [10]:
# Get the featured image and its url
images = []

di = soup.find('div', class_ = 'col-md-12 mt-5')

locates = di.find_all('div', class_='col-md-4')

for loc in locates:
    temp = {}
    try: 
        img = loc.img
        img_url = img['src'] 
        im = img_url.split(".")
        if (im[-1]) == "jpg":
            temp.update({"url": img_url})
            images.append(temp)
            #print(img_url)
        
    except AttributeError as e:
        print(e)
               

In [11]:
# Get the Mars fact table 
tables = pd.read_html(url3)

df = tables[0]
df.columns = ['items', 'data']
df.head()


,items,data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [12]:
tal = df.to_html(index = False)

In [13]:
#Visite the required url4
browser.visit(url4)
time.sleep(1)

In [14]:
 # Scrape page into Soup
html4 = browser.html
soup = bs(html4, "html.parser")

In [15]:
# build mars hemisphere title list
titles = []

dv = soup.find('div', class_ = 'collapsible results')

los = dv.find_all('div', class_='item')

for lo in los:
    try: 
        h = lo.find('div', class_ = 'description')
        title = h.a.h3.text
        if (title.split(' ')[-1]) == 'Enhanced':
            titles.append(title)
        
    except AttributeError as e:
        print(e)
               

In [16]:
#make a short title list (tiles without 'hemisphere and in low case')
short_t = []
for i in range(4):
    t = titles[i].split(' ')
    short_t.append((t[0] + '_' + t[-1]).lower())
    if i > 1:
        short_t[i] = ((t[0] + '_' + t[1] +'_'+ t[-1]).lower())
    
#short_t

In [17]:
#build dict for the images' titles and urls (urls) for sample images with smaller size, for easy loading
url_list =[]
u = 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/'
for j in range(4):
    temp = {}
    urlo = u + short_t[j]+'.'+'tif'
    urls = urlo + '/full.jpg'
    temp.update({'title':titles[j], 'href': urls})
    url_list.append(temp)
#url_list    